In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# ECG Emergency Classification: AI LifeSaver Adaptation Starter

## Introduction

This notebook demonstrates how to adapt the **AI LifeSaver** model for ECG image analysis and emergency classification. The PhysioNet ECG Image Digitization competition provides scans and photographs of paper ECG printouts, which need to be processed and analyzed.

### Key Objectives:
1. **Load and explore** sample ECG images from the competition dataset
2. **Adapt image processing techniques** from AI LifeSaver for ECG scans
3. **Classify emergency ECG patterns** (arrhythmia, heart attack risk, etc.)
4. **Generate automated advice** for detected cardiac conditions
5. **Support real-world triage** scenarios in emergency medical settings

### Model Adaptation Strategy

The AI LifeSaver model was originally designed for emergency detection and first aid advice. We'll adapt its computer vision and classification capabilities to:
- **Process ECG images** (both scanned and photographed paper printouts)
- **Extract signal features** from digitized ECG waveforms
- **Identify critical patterns** indicating cardiac emergencies
- **Provide triage recommendations** based on detected conditions

This approach bridges the gap between image-based AI emergency detection and specialized cardiac monitoring, enabling faster response in critical situations.

In [ ]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import torch
import torchvision.transforms as transforms
from pathlib import Path

print("Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# List all available input files
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:5]:  # Show first 5 files from each directory
        print(os.path.join(dirname, filename))

In [ ]:
# Delete the default first code cell - it's redundant
# This cell can be removed during cleanup

## 📸 Section 1: Loading Sample ECG Images

In this section, we'll load and visualize sample ECG images from the PhysioNet competition dataset. These images may include:
- **Scanned ECG printouts** (high quality, clear paper scans)
- **Photographed ECG printouts** (mobile phone captures, varying lighting/angles)
- **Various ECG lead configurations** (12-lead, single-lead, etc.)

In [ ]:
# Load sample ECG images from the competition dataset
data_dir = Path('/kaggle/input/physionet-ecg-image-digitization')

# Find sample images
image_files = []
if data_dir.exists():
    for ext in ['*.png', '*.jpg', '*.jpeg']:
        image_files.extend(list(data_dir.rglob(ext)))
    print(f"Found {len(image_files)} ECG image files")
    
    # Display first few sample images
    sample_count = min(3, len(image_files))
    if sample_count > 0:
        fig, axes = plt.subplots(1, sample_count, figsize=(15, 5))
        if sample_count == 1:
            axes = [axes]
        
        for idx in range(sample_count):
            img = Image.open(image_files[idx])
            axes[idx].imshow(img)
            axes[idx].set_title(f'Sample ECG {idx+1}\n{image_files[idx].name}')
            axes[idx].axis('off')
        
        plt.tight_layout()
        plt.show()
        print(f"\nDisplayed {sample_count} sample ECG images")
    else:
        print("No images found in the dataset directory")
else:
    print("Dataset directory not found. Using placeholder demonstration.")

In [ ]:
# Placeholder for image preprocessing demonstration
# This would contain ECG-specific image processing code
print("Image preprocessing pipeline:")
print("1. Perspective correction for photographed ECGs")
print("2. Contrast enhancement and noise reduction")
print("3. Grid line detection and removal")
print("4. ECG waveform extraction")

## 🔍 Section 2: AI LifeSaver Image Processing Adaptation

### Adapting AI LifeSaver for ECG Image Analysis

The **AI LifeSaver** model was designed for general emergency scene detection. To adapt it for ECG analysis, we need to modify the image processing pipeline:

#### Key Adaptations:

1. **Preprocessing for Paper ECG Images:**
   - **Perspective correction** - Handle photographed ECGs with skewed angles
   - **Contrast enhancement** - Improve visibility of faint ECG traces
   - **Noise reduction** - Remove paper artifacts and scanner imperfections
   - **Grid line detection & removal** - Extract clean ECG waveforms from gridded paper

2. **Feature Extraction Modifications:**
   - Replace general object detection with **specialized ECG waveform detection**
   - Adapt convolutional layers to recognize **QRS complexes, P waves, T waves**
   - Focus on **temporal patterns** in waveform sequences rather than spatial object detection

3. **Signal Digitization:**
   - Convert image pixels to **time-series ECG signal data**
   - Extract lead-specific information from 12-lead configurations
   - Normalize amplitudes and time scales

4. **Transfer Learning Strategy:**
   - Freeze early CNN layers (basic edge/shape detection remains useful)
   - Fine-tune middle layers for ECG-specific pattern recognition
   - Retrain classification head for cardiac emergency types

In [ ]:
# Placeholder for ECG feature extraction
# This demonstrates adapting AI LifeSaver's feature extraction for ECG signals
print("ECG Feature Extraction:")
print("- QRS complex detection")
print("- P-wave and T-wave identification")
print("- Heart rate variability analysis")
print("- ST-segment elevation/depression measurement")

In [ ]:
# Section 4: Emergency Advice Generation
# Generate automated triage recommendations based on detected patterns

def generate_emergency_advice(detected_condition, confidence):
    advice_database = {
        "STEMI": {
            "condition": "ST-Elevation Myocardial Infarction",
            "urgency": "CRITICAL - CALL 911",
            "action": "Immediate emergency medical attention required. Administer aspirin if available.",
            "details": "Acute heart attack in progress. Every minute counts for survival."
        },
        "VT": {
            "condition": "Ventricular Tachycardia",
            "urgency": "CRITICAL",
            "action": "Seek emergency care immediately. Prepare for possible defibrillation.",
            "details": "Life-threatening arrhythmia. Patient may lose consciousness."
        },
        "AFib": {
            "condition": "Atrial Fibrillation",
            "urgency": "URGENT",
            "action": "Contact cardiologist within 24 hours. Monitor for stroke symptoms.",
            "details": "Irregular rhythm increases stroke risk 5x. Requires anticoagulation evaluation."
        }
    }
    
    if detected_condition in advice_database and confidence > 0.7:
        return advice_database[detected_condition]
    return None

# Example demonstration
print("=" * 60)
print("EMERGENCY TRIAGE ADVICE SYSTEM (Simulated Output)")
print("=" * 60)
advice = generate_emergency_advice("STEMI", 0.85)
if advice:
    print(f"\nDetected Condition: {advice['condition']} (85% confidence)")
    print(f"Urgency Level: {advice['urgency']}")
    print(f"Recommended Action: {advice['action']}")
    print(f"Additional Info: {advice['details']}")
print("\n" + "=" * 60)

In [ ]:
# Placeholder: Load AI LifeSaver pretrained weights
# In practice, we would load the model from /kaggle/input/ai-lifesaver/
print("AI LifeSaver model path: /kaggle/input/ai-lifesaver/pytorch/default/1")
print("\nNext steps:")
print("1. Load pretrained weights from AI LifeSaver")
print("2. Freeze base CNN layers")
print("3. Fine-tune on ECG dataset")
print("4. Validate performance metrics")

## ❤️ Section 3: Emergency ECG Pattern Classification Plan

### Key Emergency ECG Types to Detect

Our adapted AI LifeSaver model will focus on identifying critical cardiac patterns that require immediate medical attention:

#### Target Classification Categories:

1. **Arrhythmias:**
   - Atrial Fibrillation (AFib) - Irregular rhythm, missing P waves
   - Ventricular Tachycardia (VT) - Rapid ventricular rate
   - Ventricular Fibrillation (VF) - Chaotic, life-threatening rhythm
   - Supraventricular Tachycardia (SVT)

2. **Myocardial Infarction (Heart Attack) Indicators:**
   - ST-Elevation (STEMI) - Raised ST segment indicating acute MI
   - ST-Depression - Possible ischemia or NSTEMI
   - T-wave inversions
   - Pathological Q waves

3. **Conduction Abnormalities:**
   - Bundle Branch Blocks (LBBB/RBBB)
   - AV Blocks (1st, 2nd, 3rd degree)
   - Prolonged QT interval (risk of sudden cardiac death)

4. **Other Critical Patterns:**
   - Bradycardia (dangerously slow heart rate)
   - Tachycardia (dangerously fast heart rate)
   - Premature ventricular contractions (PVCs)

### Classification Approach
We'll use a **multi-label classification** architecture that can detect multiple conditions simultaneously, as real-world ECGs often show combined pathologies.

In [ ]:
# Placeholder: ECG Classification Model (adapted from AI LifeSaver)
import torch.nn as nn

class ECGEmergencyClassifier(nn.Module):
    def __init__(self, num_classes=10):
        super(ECGEmergencyClassifier, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes),
            nn.Sigmoid()  # Multi-label classification
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = ECGEmergencyClassifier(num_classes=10)
print(f"Model initialized: {sum(p.numel() for p in model.parameters())} parameters")
print("\nTarget classes: AFib, VT, STEMI, BBB, Bradycardia, Tachycardia, QT Prolongation, AV Block, Normal, Other")